In [ ]:
import netCDF4 as nc
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
import geopandas as gpd
from pyproj import CRS, Transformer
import pandas as pd

In [ ]:
#Opening and Concatenating all files

u_wind1 = xr.open_dataset('uas_AUS-11_ERA5_historical_hres_BOM_BARRA-R2_v1_1hr_201910-201910.nc.nc4')

u_wind2 = xr.open_dataset('uas_AUS-11_ERA5_historical_hres_BOM_BARRA-R2_v1_1hr_201911-201911.nc.nc4')

u_wind  =  xr.concat([u_wind1, u_wind2], dim='time')

#u_wind = u_wind.sel(time=u_wind['time'][::4])

v_wind1 = xr.open_dataset('vas_AUS-11_ERA5_historical_hres_BOM_BARRA-R2_v1_1hr_201910-201910.nc.nc4')

v_wind2 = xr.open_dataset('vas_AUS-11_ERA5_historical_hres_BOM_BARRA-R2_v1_1hr_201911-201911.nc.nc4')

v_wind  =  xr.concat([v_wind1, v_wind2], dim='time')

#v_wind = v_wind.sel(time=v_wind['time'][::4])




rh1 = xr.open_dataset('hurs_AUS-11_ERA5_historical_hres_BOM_BARRA-R2_v1_1hr_201910-201910.nc.nc4')

rh2 = xr.open_dataset('hurs_AUS-11_ERA5_historical_hres_BOM_BARRA-R2_v1_1hr_201911-201911.nc.nc4')

rh  =  xr.concat([rh1, rh2], dim='time')

#rh = rh.sel(time=rh['time'][::4])

sm1 = xr.open_dataset('mrsos_AUS-11_ERA5_historical_hres_BOM_BARRA-R2_v1_1hr_201910-201910.nc.nc4')

sm2 = xr.open_dataset('mrsos_AUS-11_ERA5_historical_hres_BOM_BARRA-R2_v1_1hr_201911-201911.nc.nc4')

sm  =  xr.concat([sm1, sm2], dim='time')

#sm = sm.sel(time=sm['time'][::4])


st1 = xr.open_dataset('tas_AUS-11_ERA5_historical_hres_BOM_BARRA-R2_v1_1hr_201910-201910.nc.nc4')

st2 = xr.open_dataset('tas_AUS-11_ERA5_historical_hres_BOM_BARRA-R2_v1_1hr_201911-201911.nc.nc4')

st  =  xr.concat([st1, st2], dim='time')

#st = st.sel(time=st['time'][::4])


In [ ]:
#Creating Wind Speed and Direction

u = u_wind['uas']
v = v_wind['vas']

# Calculate wind speed
wind_speed = np.sqrt(u**2 + v**2)

# Calculate wind direction in radians
wind_direction_rad = np.arctan2(u, v)

# Convert wind direction to degrees
wind_direction_deg = np.degrees(wind_direction_rad)

# Normalize direction to [0, 360] degrees
wind_direction_deg = (wind_direction_deg + 360) % 360

In [ ]:
# Load the shapefile
gdf = gpd.read_file('Grid_0_150.shp')



In [ ]:
ft_layer = gpd.read_file('Fuel Layer.shp')

In [ ]:
# Define the coordinate reference systems
crs_epsg_3857 = CRS("EPSG:3857")
crs_epsg_4326 = CRS("EPSG:4326")

# Create a transformer object for EPSG:3857 to EPSG:4326
transformer = Transformer.from_crs(crs_epsg_3857, crs_epsg_4326, always_xy=True)

In [ ]:
# Extract coordinates
gdf['x'] = gdf.geometry.x
gdf['y'] = gdf.geometry.y

gdf['lon'], gdf['lat'] = transformer.transform(gdf['x'].values, gdf['y'].values)

coords = gdf[['lon', 'lat']]
time_index = u_wind['time'].to_index()

start_date = pd.to_datetime(time_index[0])
end_date = start_date + pd.Timedelta(days=18)
limited_time_index = time_index[(time_index >= start_date) & (time_index < end_date)]

In [ ]:
limited_time_index

In [ ]:
# Create the DataArray and Dataset with the limited time index
data = xr.DataArray(
    np.zeros((len(limited_time_index), len(gdf))),
    dims=['time', 'points'],
    coords={'time': limited_time_index, 'points': np.arange(len(gdf))}
)

ds = xr.Dataset(
    {
        'data': data
    },
    coords={
        'time': limited_time_index,
        'points': np.arange(len(gdf)),
        'x': ('points', gdf['lon']),
        'y': ('points', gdf['lat']), 
        'Is Fire': ('points', gdf['Is Fire']),
        'Aspect': ('points', gdf['Aspect1']),
        'Slope': ('points', gdf['Slope1']),
        'Fuel Type': ('points', ft_layer['Fuel Type1']),
        'TSF': ('points', gdf['Label'])
    }
)

subset_ds = ds

# Convert to DataFrame and reset index
df = subset_ds.to_dataframe().reset_index()

# Format the 'time' column
df['time'] = df['time'].dt.strftime('%Y-%m-%dT%H:%M:%S.000000000')




In [ ]:
df['Is Fire'] = df['Is Fire'].fillna(0)


In [ ]:
df['TSF'] = df['TSF'].apply(lambda x: '>10' if x == '2015-2016 Prescribed Burn' else '<10')


In [ ]:
# Function to get the wind speed for a given time and location
def get_weather_at_location_and_time(var, time, lon, lat):
    # Select the wind speed value for the given time, longitude, and latitude
    #print("time", time)
    return var.sel(time=time, lon=lon, lat=lat, method='nearest').item()



df['wspd'] = df.apply(lambda row: get_weather_at_location_and_time(wind_speed, row['time'], row['x'], row['y']), axis=1)
print("done")
df['wind_direction_deg'] = df.apply(lambda row: get_weather_at_location_and_time(wind_direction_deg, row['time'], row['x'], row['y']), axis=1)
print("done")
df['rh'] = df.apply(lambda row: get_weather_at_location_and_time(rh['hurs'], row['time'], row['x'], row['y']), axis=1)
print("done")

df['sm'] = df.apply(lambda row: get_weather_at_location_and_time(sm['mrsos'], row['time'], row['x'], row['y']), axis=1)
print("done")

df['st'] = df.apply(lambda row: get_weather_at_location_and_time(st['tas'], row['time'], row['x'], row['y']), axis=1)
print("done")


In [ ]:
gdf1 = gpd.read_file('Grid_1_150.shp')
gdf2 = gpd.read_file('Grid_2 150.shp')
gdf3 = gpd.read_file('Grid_3_150.shp')
gdf4 = gpd.read_file('Grid_4_150.shp')
gdf5 = gpd.read_file('Grid_5_150.shp')
gdf6 = gpd.read_file('Grid_6_150.shp')
gdf7 = gpd.read_file('Grid_7_150.shp')
gdf8 = gpd.read_file('Grid_8_150.shp')
gdf9 = gpd.read_file('Grid_9_150.shp')



In [ ]:
#Timestep 0

target_time = pd.to_datetime('2019-10-26 05:51:24') 
df['time'] = pd.to_datetime(df['time'])

# Step 2: Find closest timestep in df to target_time
closest_timestep = df.loc[df['time'].sub(target_time).abs().idxmin(), 'time']

# Step 3: Replace target_time with closest_timestep in the DataFrame
df.loc[df['time'] == target_time, 'time'] = closest_timestep

closest_timestep


In [ ]:
threshold_time = closest_timestep

# Filter rows where 'time' is greater than or equal to threshold_time
df = df[df['time'] >= threshold_time]

In [ ]:
#Timestep 1

target_time = pd.to_datetime('2019-10-28 13:00:00') 
df['time'] = pd.to_datetime(df['time'])

# Step 2: Find closest timestep in df to target_time
closest_timestep = df.loc[df['time'].sub(target_time).abs().idxmin(), 'time']

# Step 3: Replace target_time with closest_timestep in the DataFrame
df.loc[df['time'] == target_time, 'time'] = closest_timestep

df.loc[df['time'] == closest_timestep, 'Is Fire'] = gdf1['Is Fire'].values


print(closest_timestep)

In [ ]:
#Timestep 2

target_time = pd.to_datetime('2019-11-01 13:00:00') 

df['time'] = pd.to_datetime(df['time'])

# Step 2: Find closest timestep in df to target_time
closest_timestep = df.loc[df['time'].sub(target_time).abs().idxmin(), 'time']

# Step 3: Replace target_time with closest_timestep in the DataFrame
df.loc[df['time'] == target_time, 'time'] = closest_timestep

df.loc[df['time'] == closest_timestep, 'Is Fire'] = gdf2['Is Fire'].values


print(closest_timestep)

In [ ]:
#Timestep 3

target_time = pd.to_datetime('2019-11-02 07:00:00')

df['time'] = pd.to_datetime(df['time'])

# Step 2: Find closest timestep in df to target_time
closest_timestep = df.loc[df['time'].sub(target_time).abs().idxmin(), 'time']

# Step 3: Replace target_time with closest_timestep in the DataFrame
df.loc[df['time'] == target_time, 'time'] = closest_timestep

df.loc[df['time'] == closest_timestep, 'Is Fire'] = gdf3['Is Fire'].values


print(closest_timestep)

In [ ]:
#Timestep 4

target_time = pd.to_datetime('2019-11-04 04:00:00')  

df['time'] = pd.to_datetime(df['time'])

# Step 2: Find closest timestep in df to target_time
closest_timestep = df.loc[df['time'].sub(target_time).abs().idxmin(), 'time']

# Step 3: Replace target_time with closest_timestep in the DataFrame
df.loc[df['time'] == target_time, 'time'] = closest_timestep

df.loc[df['time'] == closest_timestep, 'Is Fire'] = gdf4['Is Fire'].values


print(closest_timestep)

In [ ]:
#Timestep 5

target_time = pd.to_datetime('2019-11-06 20:00:00') 
df['time'] = pd.to_datetime(df['time'])

# Step 2: Find closest timestep in df to target_time
closest_timestep = df.loc[df['time'].sub(target_time).abs().idxmin(), 'time']

# Step 3: Replace target_time with closest_timestep in the DataFrame
df.loc[df['time'] == target_time, 'time'] = closest_timestep

df.loc[df['time'] == closest_timestep, 'Is Fire'] = gdf5['Is Fire'].values


print(closest_timestep)

In [ ]:
#Timestep 6

target_time = pd.to_datetime('2019-11-07 03:30:00') 

df['time'] = pd.to_datetime(df['time'])

# Step 2: Find closest timestep in df to target_time
closest_timestep = df.loc[df['time'].sub(target_time).abs().idxmin(), 'time']

# Step 3: Replace target_time with closest_timestep in the DataFrame
df.loc[df['time'] == target_time, 'time'] = closest_timestep

df.loc[df['time'] == closest_timestep, 'Is Fire'] = gdf6['Is Fire'].values


print(closest_timestep)

In [ ]:
#Timestep 7

target_time = pd.to_datetime('2019-11-07 21:00:00')  
df['time'] = pd.to_datetime(df['time'])

# Step 2: Find closest timestep in df to target_time
closest_timestep = df.loc[df['time'].sub(target_time).abs().idxmin(), 'time']

# Step 3: Replace target_time with closest_timestep in the DataFrame
df.loc[df['time'] == target_time, 'time'] = closest_timestep

df.loc[df['time'] == closest_timestep, 'Is Fire'] = gdf7['Is Fire'].values


print(closest_timestep)

In [ ]:
#Timestep 8

target_time = pd.to_datetime('2019-11-08 02:00:00') 

df['time'] = pd.to_datetime(df['time'])

# Step 2: Find closest timestep in df to target_time
closest_timestep = df.loc[df['time'].sub(target_time).abs().idxmin(), 'time']

# Step 3: Replace target_time with closest_timestep in the DataFrame
df.loc[df['time'] == target_time, 'time'] = closest_timestep

df.loc[df['time'] == closest_timestep, 'Is Fire'] = gdf8['Is Fire'].values


print(closest_timestep)

In [ ]:
#Timestep 9

target_time = pd.to_datetime('2019-11-08 15:00:00')  

df['time'] = pd.to_datetime(df['time'])

# Step 2: Find closest timestep in df to target_time
closest_timestep = df.loc[df['time'].sub(target_time).abs().idxmin(), 'time']

# Step 3: Replace target_time with closest_timestep in the DataFrame
df.loc[df['time'] == target_time, 'time'] = closest_timestep

df.loc[df['time'] == closest_timestep, 'Is Fire'] = gdf9['Is Fire'].values


print(closest_timestep)

In [ ]:
'''
target_timesteps = ('2019-10-26 06:00:00','2019-10-28 13:00:00', '2019-11-01 13:00:00','2019-11-02 07:00:00',
                   '2019-11-04 04:00:00','2019-11-06 20:00:00', '2019-11-07 03:30:00', 
                   '2019-11-07 21:00:00','2019-11-08 02:00:00', '2019-11-08 15:00:00')
                   
'''

In [ ]:
#df = df[df['time'].isin(target_timesteps)]


In [ ]:
df['time'].unique()

In [ ]:
# Step 1: Convert 'time' column to datetime format if it's not already
df['time'] = pd.to_datetime(df['time'])

# Step 2: Convert 'time' column to desired string format
df['time'] = df['time'].dt.strftime('%Y-%m-%d %H:%M:%S')

In [ ]:
df

In [ ]:
df.to_csv("Port_Fire_full.csv", index=False)  # Set index=False to exclude row numbers


In [ ]:
print("all good!")